In [ ]:

import numpy as np
import math

import pickle
from PIL import Image
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from scipy import stats
from numpy.random import randint,randn,standard_normal,random_sample
from plotly.offline import plot

from matplotlib import pyplot as plt
logmin=-.85
import plotly.graph_objects as go
def funcsig(w,X,b): # sigmoid functions
    return 1/(1+np.exp(-np.matmul(w.transpose(),X)+b))





In [ ]:
def lossFun(w,M,X,b,lam):
    yi=X[:,-1]
    X=X[:,:-1].transpose()
    h=funcsig(w,X,b)
    for val in h:
        if(val>logmax):
            val=logmax
        if(val<logmin):
            val=logmin
    reg=(lam/2)*np.matmul(w.transpose(),w)
    L= -(1/M)*np.sum((np.multiply(yi,np.log(h)))+(np.multiply(1-yi,np.log(1-h)))) + reg
    return L
    


    
    

In [ ]:
def grayscale(data, dtype='float32'):
    # luma coding weighted average in video systems
    r, g, b = np.asarray(.3, dtype=dtype), np.asarray(.59, dtype=dtype), np.asarray(.11, dtype=dtype)
    rst = r * data[:, :, :, 0] + g * data[:, :, :, 1] + b * data[:, :, :, 2]
    # add channel dimension
    rst = np.expand_dims(rst, axis=3)
    return rst


In [ ]:
def unpickle(file):
    with open(file,'rb') as fo:
        dict= pickle.load(fo,encoding ='latin1')
    return dict


In [ ]:
def gradient(w,M,X,b,lam):
    yi=X[:,-1]
    X=X[:,:-1].transpose()
    h=funcsig(w,X,b)
    for val in h:
        if(val>logmax):
            val=logmax
        if(val<logmin):
            val=logmin
    X=X.transpose()
    return -(1./M)*np.matmul((yi-h).transpose(),X)+(lam*w)
    
    

In [ ]:
def gradientb(w,M,X,b,lam):
    yi=X[:,-1]
    X=X[:,:-1].transpose()
    h=funcsig(w,X,b)
    return np.sum(yi-h)/M
    

In [ ]:
def test_sample(w,X,b,weightclass):
    yi=X[-1]
    X=X[:-1].transpose()
    h=funcsig(w,X,b)
    return h
    

In [ ]:
def gradientDescent(w,M,X,b,iters,lr,lam):
    for x in range(iters):
        L=lossFun(w,M,X,b,lam)
        G=gradient(w,M,X,b,lam)
        B=gradientb(w,M,X,b,lam)
        w=w-(lr*G)
        b=b-(lr*B)
        print("Loss After GD Update "+ str(x) + " is " + str(L))
    return w,b


    

In [ ]:
def linClass(X,Y):
    w=np.random.standard_normal(1024)
    b=random_sample()
    X=np.concatenate((X,Y.T.reshape(50000,1)),axis=1)
    w,b=gradientDescent(w,50000,X,b,100,10,1e-2)
    return w,b
    #print(w)## loss becomes around .3333


In [ ]:

b1=unpickle('cifar-10-batches-py/data_batch_1')
b2=unpickle('cifar-10-batches-py/data_batch_2')
b3=unpickle('cifar-10-batches-py/data_batch_3')
b4=unpickle('cifar-10-batches-py/data_batch_4')
b5=unpickle('cifar-10-batches-py/data_batch_5')
meta=unpickle('cifar-10-batches-py/batches.meta')
tb=unpickle('cifar-10-batches-py/test_batch')

label_names=['label_names']

join_data= np.concatenate((b1['data'],b2['data'],b3['data'],b4['data'],b5['data']))
join_labels= np.concatenate((b1['labels'],b2['labels'],b3['labels'],b4['labels'],b5['labels']))
#
#test_data= np.concatenate((tb['data']))
#test_labels=np.concatenate((tb['labels']))


test_data= tb['data']
test_labels=tb['labels']

join_datargb=join_data.reshape(50000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
join_datagray=grayscale(join_datargb).reshape(50000,1024)
#join_data_normalized=stats.zscore(join_datagray,axis=1)  
join_data_normalized=normalize(join_datagray,axis=1)  

#print(join_data_normalized)
    
        
# X_folds=np.array_split(join_data_normalized,5) # split into five folds
# y_folds=np.array_split(join_labels,5)
linelist=[]
for i in range(0,10):
    new_labels=np.copy(join_labels)
    for c,v in enumerate(new_labels):
        if(v==i):
            new_labels[c]=1
        else:
            new_labels[c]=0
    print("Traning Classifier with class "+ str(i))
    w,b=linClass(join_data_normalized,new_labels)
    linelist.append([w,b])
            
    
            
            

#print(join_data_normalized)

In [ ]:
for c,wvs in enumerate(linelist):
    imgdata=wvs[0].reshape(32,32)
    print(imgdata)
    #imgdata=imgdata/
    plt.imshow(imgdata)
    plt.savefig('classifier'+str(c)+'.jpg')
    #img = Image.fromarray(imgdata,mode='L')
    #img.show()
    #img.save('classifier'+str(c)+'.jpg')
    #img.show()
    

    
    

In [ ]:

test_data= tb['data']
test_labels=np.array(tb['labels'])

test_datargb=test_data.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("uint8")
test_datagray=grayscale(test_datargb).reshape(10000,1024)
X=normalize(test_datagray,axis=1)
probvect=[]
for samp in X:# loop thrhough Test examples
    probclass=[]
    #print(X.shape)
    for c,wvs in enumerate(linelist): #loop through all weight vectors
        wv=np.array(wvs[0])
        b=wvs[1]
        prob=funcsig(wv,samp,b)
        probclass.append(prob)
    probvect.append(probclass)
    
print(probvect)
    

    
   

        

    
    

In [ ]:
classification_accuracies=[]
print(len(linelist))
for c,wvs in enumerate(linelist):#loop through all weight vectors
        corrclass=0
        wv=np.array(wvs[0])
        b=wvs[1]
        for count,samp in enumerate(X):
            prob=funcsig(wv,samp,b)
            #print(prob)
            if(prob>.49999999 and test_labels[count]==c):##correctly guesss this class
                corrclass+=1
            elif(prob<.49999999 and test_labels[count]!=c):
                corrclass+=1
        classification_accuracies.append(corrclass/10000)
print(classification_accuracies)
        
    


In [ ]:
cm=(10,10)
cm=np.zeros(cm)
print(cm)
for sam,probs in enumerate(probvect):
    cm[test_labels[sam],np.argmax(probs)]+=1
print(cm)

accuracy=np.sum(cm.diagonal())/100 # diagonal of array is correctly classified example

print("Total Accuracy is "+ str(accuracy) +"%")

    